# pytesseract

In [12]:
from PIL import Image
import pytesseract
import configparser
import os
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [17]:
print(pytesseract.image_to_string('ocrtest.jpg',lang='kor+eng'))

기능성 화장품 사용 팁

면 좋은 SET AE
전용 화장품과세라미이드
~AHA(QITSIOIE SAIS)
.8804메타하이드록시샌 와 BSR
-모공관리제품과 OAS
-비타민화 비타민
-얄부틴과비타민6
(20K HIERN Hoke: ESOL AHA
=O) NB 뒤에 BENE AS
eee ny
~ AHAGBHAS! Elis
Aleta} HIE}2IC
~ AHASH 비타민6
-모공조절저품
AO} SAE 안에이징제품
= e/a] 비타민
~The 보슴제품
자료:식품의약품안전처


# tesserocr

In [19]:
import tesserocr

# with tesserocr.PyTessBaseAPI(lang='eng+kor') as api:
#     api.SetImageFile('eSXSz.jpg')
#     print api.GetUTF8Text()

# or simply
print(tesserocr.file_to_text('ocrtest.jpg', lang='eng'))

7is8 383 NSE

oi $2 SET AE
Bg si5a} AIOE
~AHA(QITSIOIE SAIS)
ABHAIHIEHS}OI SAlA)04 BSR
- Bape AEH OAS
~BE}BICS} BIE}
~ FEIT BEBIC
(20K HIERN Hoke: ESOL AHA
=O) NB 0] BENE AS
eee ny
~ AHAGBHAS! Elis
Aleta} HIE}2IC
~ AHASH BIEHIC
-BERENE
AO} SAE clelo|OVS Hs
= e/a] HIEIRIC
~The BOM
ASAZINEA

 

  

 

 



In [25]:
from tesserocr import PyTessBaseAPI

images = ['ocrtest.jpg']

with PyTessBaseAPI() as api:
    for img in images:
        api.SetImageFile(img)
        print(api.GetUTF8Text())
        print(api.AllWordConfidences())
# api is automatically finalized when used in a with-statement (context manager).
# otherwise api.End() should be explicitly called when it's no longer needed.

7is8 383 NSE

oi $2 SET AE
Bg si5a} AIOE
~AHA(QITSIOIE SAIS)
ABHAIHIEHS}OI SAlA)04 BSR
- Bape AEH OAS
~BE}BICS} BIE}
~ FEIT BEBIC
(20K HIERN Hoke: ESOL AHA
=O) NB 0] BENE AS
eee ny
~ AHAGBHAS! Elis
Aleta} HIE}2IC
~ AHASH BIEHIC
-BERENE
AO} SAE clelo|OVS Hs
= e/a] HIEIRIC
~The BOM
ASAZINEA

 

  

 

 

[31, 50, 33, 38, 39, 38, 57, 25, 0, 10, 0, 0, 0, 0, 8, 16, 0, 38, 17, 0, 20, 42, 25, 0, 25, 11, 40, 36, 66, 0, 79, 56, 55, 62, 36, 26, 73, 60, 62, 19, 0, 53, 0, 33, 0, 2, 0, 0, 35, 54, 31, 0, 2, 0, 0, 95, 95, 95, 95]


In [24]:
print(tesserocr.get_languages())

('C:\\Users\\ICT01_20\\Anaconda3\\envs\\tf_test/tessdata/', ['eng', 'kor', 'kor_vert', 'osd'])


In [3]:
import tesserocr 
a = baseApi.init(r'C:\Users\ICT01_20\Documents\BM_YounghoonKANG_Python', lang='eng')

ModuleNotFoundError: No module named 'tesserocr'

# Kakao REST API

In [2]:
import json
import cv2
import requests
import sys

LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40

def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)
        return image_path
    return None

def kakao_ocr_detect(image_path: str, appkey: str):
    """
    detect api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/detect'
    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}
    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()
    return requests.post(API_URL, headers=headers, files={"file": data})

def kakao_ocr_recognize(image_path: str, boxes: list, appkey: str):
    """
    recognize api request example
    :param boxes: 감지된 영역 리스트. Canvas 좌표계: 좌상단이 (0,0) / 우상단이 (limit,0)
                    감지된 영역중 좌상단 점을 기준으로 시계방향 순서, 좌상->우상->우하->좌하
                    ex) [[[0,0],[1,0],[1,1],[0,1]], [[1,1],[2,1],[2,2],[1,2]], ...]
    :param image_path: 이미지 파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/recognize'
    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}
    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()
    return requests.post(API_URL, headers=headers, files={"file": data}, data={"boxes": json.dumps(boxes)})

def main():
    if len(sys.argv) != 3:
        print("Please run with args: $ python example.py /path/to/image appkey")
    #image_path, appkey = sys.argv[1], sys.argv[2]
    image_path, appkey = "ocrtest.jpg", "5dc4d50e5b850c0b040d5c308ef6e3c4"
    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")
    output = kakao_ocr_detect(image_path, appkey).json()
    print("[detect] output:\n{}\n".format(output))
    boxes = output["result"]["boxes"]
    boxes = boxes[:min(len(boxes), LIMIT_BOX)]
    output = kakao_ocr_recognize(image_path, boxes, appkey).json()
    print("[recognize] output:\n{}\n".format(json.dumps(output, sort_keys=True, indent=2, ensure_ascii=False)))

if __name__ == "__main__":
    main()

[detect] output:
{'result': {'boxes': [[[56, 75], [90, 75], [90, 91], [56, 91]], [[97, 75], [130, 75], [130, 91], [97, 91]], [[136, 75], [159, 75], [159, 91], [136, 91]], [[163, 75], [173, 75], [173, 91], [163, 91]], [[70, 105], [90, 105], [90, 118], [70, 118]], [[94, 105], [113, 105], [113, 117], [94, 117]], [[119, 105], [140, 105], [140, 118], [119, 118]], [[145, 105], [176, 105], [176, 118], [145, 118]], [[180, 105], [201, 105], [201, 118], [180, 118]], [[64, 127], [94, 127], [94, 139], [64, 139]], [[100, 127], [119, 127], [119, 140], [100, 140]], [[124, 127], [167, 127], [167, 139], [124, 139]], [[170, 126], [223, 126], [223, 139], [170, 139]], [[64, 145], [179, 145], [179, 157], [64, 157]], [[63, 160], [242, 160], [242, 175], [63, 175]], [[64, 179], [105, 179], [105, 192], [64, 192]], [[110, 179], [140, 179], [140, 192], [110, 192]], [[146, 179], [186, 179], [186, 192], [146, 192]], [[64, 216], [106, 216], [106, 230], [64, 230]], [[64, 199], [113, 199], [113, 212], [64, 212]], [[1